In [85]:
import pandas as pd 
from pull_covid_data import pull_covid_data


pull_covid_data()

def load_to_memory(filename):
    data = pd.read_csv(f'{filename}', header=0) 
    return data

#Replace with testing file
time_series = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
dataframe = load_to_memory(time_series)

def use_melt(df):
    melt_df = pd.melt(df, id_vars=['Country/Region'], value_vars=list(df.columns[4:]))
    melt_df.columns = ['Country/Region', 'Date', 'Cases']
    
    #Grouping by Country/Region and adding column "Cumulative Sum" as a result of groupby
    melt_df['Cumulative Sum'] = melt_df.groupby(['Country/Region'])['Cases'].cumsum(axis=0)
    
    #Changing date to proper format
    melt_df['Date'] = pd.to_datetime(melt_df['Date'])
    
    # 279 is the number of unique Country/Regions for the DataFrame
    return melt_df.tail(279)
    
use_melt(dataframe)


Here is the status update of COVID-19: 

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


,Country/Region,Date,Cases,Cumulative Sum
148149,Afghanistan,2021-07-06,127464,20742904
148150,Albania,2021-07-06,132544,24802238
148151,Algeria,2021-07-06,142447,32820101
148152,Andorra,2021-07-06,13991,2905567
148153,Angola,2021-07-06,39375,6178651
...,...,...,...,...
148423,Vietnam,2021-07-06,22341,1103416
148424,West Bank and Gaza,2021-07-06,314780,53675169
148425,Yemen,2021-07-06,6931,1222752
148426,Zambia,2021-07-06,167132,18280206


In [80]:
import unittest
import pandas as pd

class TestAnalysis(unittest.TestCase):
    def test_cumulative_sum(self):
        test_file_name = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        df = load_to_memory(test_file_name)
        use_melt(df)
        print(use_melt(df))
        
        self.assertTrue(df[df['Country/Region'] == 'Afghanistan'] & df[df['Date'] == ['2020-07-06']]) == 20742904
#         self.assertEqual(df[df['Country/Region'] == 'Zimbabwe'], 0)
#             & df['Date'] == ['2020-08-13'] == 500000
        
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_cumulative_sum (__main__.TestAnalysis) ... 

Using Melt Function
Using Melt Function
            Country/Region       Date   Cases  Cumulative Sum
148149         Afghanistan 2021-07-06  127464        20742904
148150             Albania 2021-07-06  132544        24802238
148151             Algeria 2021-07-06  142447        32820101
148152             Andorra 2021-07-06   13991         2905567
148153              Angola 2021-07-06   39375         6178651
...                    ...        ...     ...             ...
148423             Vietnam 2021-07-06   22341         1103416
148424  West Bank and Gaza 2021-07-06  314780        53675169
148425               Yemen 2021-07-06    6931         1222752
148426              Zambia 2021-07-06  167132        18280206
148427            Zimbabwe 2021-07-06   57963         8142767

[279 rows x 4 columns]


ERROR

ERROR: test_cumulative_sum (__main__.TestAnalysis)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Library/Python/3.8/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Date'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-80-f426da9e68f5>", line 11, in test_cumulative_sum
    self.assertTrue(df[df['Country/Region'] == 'Afghanistan'] & df[df['Date'] == ['2020-07-06']]) == 207

In [98]:
csv_file = 'https://raw.githubusercontent.com/outbreak-info/JHU-CSSE/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
deaths_file = 'https://raw.githubusercontent.com/outbreak-info/JHU-CSSE/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

def create_test_file(filename):
    data = pd.read_csv(f'{filename}', header=0) 
    edited_data = data.iloc[0, :7]
    return edited_data

create_test_file(csv_file)


def create_deaths_file(filename):
    data = pd.read_csv(f'{filename}', header=0)
    edited_data = data.iloc[0, 4:7]
    return edited_data
    
def join_death_data():
    new_file = create_test_file(csv_file).append(create_deaths_file(deaths_file))
    return new_file
    
join_death_data()


Province/State            NaN
Country/Region    Afghanistan
Lat                  33.93911
Long                67.709953
1/22/20                     0
1/23/20                     0
1/24/20                     0
1/22/20                     0
1/23/20                     0
1/24/20                     0
Name: 0, dtype: object